<a href="https://colab.research.google.com/github/BRomans/IdMind/blob/main/eeg_biometrics_autoencoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pytorch_lightning

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from google.colab import drive
import os
import pickle
from pathlib import Path
from matplotlib import pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
import pytorch_lightning as pl
from torch.utils.data import Dataset, DataLoader

In [ ]:
drive.mount("/content/drive")
os.chdir('/content/drive/MyDrive/ml2-eeg-biometrics/prep-data')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('File_explaination.csv')
print(df)
files = df.get('Filename')
print(files[8384])
print("Viva Italia")

In [ ]:
class RightHandTrain(Dataset):

  def __init__(self, train_split):
    df = pd.read_csv('File_explaination.csv')
    self.files = df.get('Filename')
    self.train_split = train_split
    self.X = [None] * int(len(self.files)*self.train_split)

  def __len__(self):
    return int(len(self.files)*self.train_split)

  def __getitem__(self, idx):
    if self.X[idx] is None:
      self.X[idx] = torch.load(self.files[idx])

    return self.X[idx]

class RightHandVal(Dataset):

  def __init__(self, val_split):
    df = pd.read_csv('File_explaination.csv')
    self.files = df.get('Filename')
    self.val_split = val_split
    self.X = [None] * (int(len(self.files)*self.val_split)-1)

  def __len__(self):
    return int(len(self.files)*self.val_split) -1

  def __getitem__(self, idx):
    idx += int(len(self.files) * (1 - self.val_split)) +1 
    if self.X[idx] is None:
      self.X[idx] = torch.load(self.files[idx])

    return self.X[idx]

In [ ]:
ds_train = RightHandTrain(0.6)
ds = RightHandTrain(0.6)
#x_train, x_test = train_test_split(ds)
len(ds)


In [ ]:
x = torch.load('./A4|0|0|0.pt')
x = x.view(1,29,2500)
x = x.float()
print(x.size())

In [ ]:
class Autoencoder(pl.LightningModule):

  def __init__(self):
    super().__init__()
      
    # encoder layers 
    self.conv1 = nn.Conv1d(29, 50, 5)  
    self.conv2 = nn.Conv1d(50, 20, 5)
    self.conv3 = nn.Conv1d(20, 5, 5)

    # decoder layers
    self.t_conv1 = nn.ConvTranspose1d(5, 20, 5)
    self.t_conv2 = nn.ConvTranspose1d(20, 50, 5)
    self.t_conv3 = nn.ConvTranspose1d(50, 29, 5)

    # pooling layer to reduce dims by two
    #self.pool = nn.MaxPool1d(2)

    # activation function
    self.act = nn.LeakyReLU(0.1)

    self.loss = nn.MSELoss()

  def forward(self, x):
    # Encode  
    x = self.act(self.conv1(x))
    x = self.act(self.conv2(x))
    x = self.act(self.conv3(x))

    # Decode
    x = self.act(self.t_conv1(x))
    x = self.act(self.t_conv2(x))
    x = self.act(self.t_conv3(x))
    return x

  def training_step(self, batch, batch_idx):
    x = batch
    x_hat = self(x)
    loss = self.loss(x_hat, x)
    return loss

  def configure_optimizers(self):
    return torch.optim.Adam(self.parameters(), lr=0.02)

In [ ]:
ae = Autoencoder()
print(ae(x).size())
